# 老虎证券真实交易API完整演示

本Notebook演示老虎证券API的所有主要功能，包括真实交易操作。

⚠️ **重要提醒**: 本演示包含真实交易操作，请谨慎使用！

## 目录
1. [环境配置与连接测试](#1-环境配置与连接测试)
2. [行情数据接口](#2-行情数据接口)
3. [账户信息接口](#3-账户信息接口)
4. [真实交易接口](#4-真实交易接口)
5. [期权数据接口](#5-期权数据接口)
6. [推送服务接口](#6-推送服务接口)
7. [高级功能演示](#7-高级功能演示)

## 1. 环境配置与连接测试

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
import time

# 设置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("环境配置完成")

环境配置完成


In [3]:
# 导入老虎证券API
try:
    from tigeropen.tiger_open_config import TigerOpenClientConfig
    from tigeropen.common.consts import Market, Currency, Language, TimelinePeriod
    from tigeropen.common.consts import BarPeriod, QuoteRight, OrderType, OrderStatus
    from tigeropen.quote.quote_client import QuoteClient
    from tigeropen.trade.trade_client import TradeClient
    from tigeropen.push.push_client import PushClient
    
    print("✅ 老虎证券API导入成功")
except ImportError as e:
    print(f"❌ 导入失败: {e}")
    sys.exit(1)

✅ 老虎证券API导入成功


e:\01_lixin_work_space\01_code\tiger_trade\.venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tigeropen/push/pb/SocketCommon.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
e:\01_lixin_work_space\01_code\tiger_trade\.venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tigeropen/push/pb/Request.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
e:\01_lixin_work_space\01_code\tiger_trade\.venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tigeropen/push/pb/Response.proto. Please update the ge

In [4]:
# 配置文件路径
config_file_path = '../config/tiger_openapi_config.properties'
private_key_path = '../config/private_key.pem'

# 检查配置文件
if not os.path.exists(config_file_path):
    print(f"❌ 配置文件不存在: {config_file_path}")
elif not os.path.exists(private_key_path):
    print(f"❌ 私钥文件不存在: {private_key_path}")
else:
    print("✅ 配置文件检查通过")

✅ 配置文件检查通过


In [5]:
# 导入必要的工具函数
from tigeropen.common.util.signature_utils import read_private_key
from tigeropen.common.consts import Language

# 初始化客户端配置
client_config = TigerOpenClientConfig(
    sandbox_debug=False,  # 生产环境
    props_path=config_file_path
)
client_config.private_key = read_private_key(private_key_path)
client_config.language = Language.zh_CN
client_config.timeout = 60

# 创建客户端实例
quote_client = QuoteClient(client_config)
trade_client = TradeClient(client_config)

# 推送客户端初始化
try:
    # 从client_config获取socket连接参数
    protocol, host, port = client_config.socket_host_port
    push_client = PushClient(host, port, use_ssl=(protocol == 'ssl'))
    print("✅ 推送客户端初始化成功")
except Exception as e:
    print(f"⚠️ 推送客户端初始化失败: {e}")
    push_client = None

print("✅ 客户端初始化完成")

2025-08-13 10:07:43,189 - INFO - sdk version: 3.4.4
2025-08-13 10:07:43,359 - INFO - Grab quote permission. Permissions:[{'name': 'hkStockQuoteLv2', 'expire_at': -1}, {'name': 'aStockQuoteLv1', 'expire_at': -1}, {'name': 'usQuoteBasic', 'expire_at': 1764739059000}, {'name': 'usOptionQuote', 'expire_at': 1770189404000}]


✅ 推送客户端初始化成功
✅ 客户端初始化完成


In [6]:
# 连接测试
try:
    # 测试行情连接 - 获取行情权限
    permissions = quote_client.grab_quote_permission()
    print(f"✅ 行情连接成功，权限信息: {permissions}")
    
    # 测试交易连接 - 获取账户信息
    try:
        accounts = trade_client.get_managed_accounts()
        print(f"✅ 交易连接成功，账户数量: {len(accounts) if accounts else 0}")
    except Exception as trade_e:
        print(f"⚠️ 交易连接测试失败: {trade_e}")
    
except Exception as e:
    print(f"❌ 行情连接测试失败: {e}")

print("🎉 基础连接测试完成！")

✅ 行情连接成功，权限信息: [{'name': 'hkStockQuoteLv2', 'expire_at': -1}, {'name': 'aStockQuoteLv1', 'expire_at': -1}, {'name': 'usQuoteBasic', 'expire_at': 1764739059000}, {'name': 'usOptionQuote', 'expire_at': 1770189404000}]
✅ 交易连接成功，账户数量: 3
🎉 基础连接测试完成！


## 2. 行情数据接口

### 2.1 获取股票基本信息

In [7]:
# 获取股票基本信息
symbols = ['AAPL', 'TSLA', 'QQQ', 'SPY']

try:
    # get_stock_briefs返回DataFrame格式数据
    stock_briefs = quote_client.get_stock_briefs(symbols)
    print("📊 股票基本信息:")
    if stock_briefs is not None and not stock_briefs.empty:
        # 遍历DataFrame的每一行
        for index, row in stock_briefs.iterrows():
            symbol = row.get('symbol', 'N/A')
            latest_price = row.get('latest_price', 0)
            pre_close = row.get('pre_close', 0)
            volume = row.get('volume', 0)
            print(f"  {symbol}: 最新价=${latest_price:.2f}, 昨收=${pre_close:.2f}, 成交量={volume:,}")
    else:
        print("  未获取到股票信息")
except Exception as e:
    print(f"❌ 获取股票信息失败: {e}")

📊 股票基本信息:
  AAPL: 最新价=$229.65, 昨收=$227.18, 成交量=55,672,301
  TSLA: 最新价=$340.84, 昨收=$339.03, 成交量=80,690,086
  QQQ: 最新价=$580.05, 昨收=$572.85, 成交量=42,271,441
  SPY: 最新价=$642.69, 昨收=$635.92, 成交量=64,821,797


### 2.2 获取实时报价

In [8]:
# 获取实时报价
try:
    # get_stock_briefs可以获取实时报价信息，返回DataFrame格式
    stock_details = quote_client.get_stock_briefs(symbols)
    print("💰 实时报价:")
    if stock_details is not None and not stock_details.empty:
        # 遍历DataFrame的每一行
        for index, row in stock_details.iterrows():
            symbol = row.get('symbol', 'N/A')
            latest_price = row.get('latest_price', 0)
            pre_close = row.get('pre_close', 0)
            # 计算涨跌幅
            if pre_close > 0:
                change = latest_price - pre_close
                change_rate = change / pre_close
            else:
                change = 0
                change_rate = 0
            print(f"  {symbol}: 最新价=${latest_price:.2f}, 涨跌={change:.2f}({change_rate:.2%})")
    else:
        print("  未获取到报价信息")
except Exception as e:
    print(f"❌ 获取报价失败: {e}")

💰 实时报价:
  AAPL: 最新价=$229.65, 涨跌=2.47(1.09%)
  TSLA: 最新价=$340.84, 涨跌=1.81(0.53%)
  QQQ: 最新价=$580.05, 涨跌=7.20(1.26%)
  SPY: 最新价=$642.69, 涨跌=6.77(1.06%)


### 2.3 获取K线数据

In [9]:
# 获取K线数据
symbol = 'AAPL'
end_time = int(time.time() * 1000)
begin_time = end_time - 30 * 24 * 60 * 60 * 1000  # 30天前

try:
    # 日K线 - get_bars返回DataFrame格式
    bars = quote_client.get_bars(
        symbols=[symbol],
        period=BarPeriod.DAY,
        begin_time=begin_time,
        end_time=end_time,
        right=QuoteRight.BR
    )
    
    if bars is not None and not bars.empty:
        # bars已经是DataFrame格式，直接处理时间列
        bars_copy = bars.copy()
        bars_copy['time'] = pd.to_datetime(bars_copy['time'], unit='ms')
        
        print(f"📈 {symbol} 日K线数据 (最近5天):")
        # 显示最近5天的数据
        recent_data = bars_copy[['time', 'open', 'high', 'low', 'close', 'volume']].tail()
        print(recent_data.to_string(index=False))
    else:
        print("❌ 未获取到K线数据")
        
except Exception as e:
    print(f"❌ 获取K线数据失败: {e}")

📈 AAPL 日K线数据 (最近5天):
               time    open   high    low  close    volume
2025-08-06 04:00:00 205.630 215.38 205.59 213.25 108483103
2025-08-07 04:00:00 218.875 220.85 216.58 220.03  90224834
2025-08-08 04:00:00 220.830 231.00 219.25 229.35 113853967
2025-08-11 04:00:00 227.920 229.56 224.76 227.18  61806132
2025-08-12 04:00:00 228.005 230.80 227.07 229.65  55672706


### 2.4 获取分时数据

In [10]:
# 获取分时数据
try:
    timeline = quote_client.get_timeline(
        symbols=[symbol],
        period=TimelinePeriod.DAY,
        begin_time=begin_time
    )
    
    if timeline is not None and not timeline.empty:
        print(f"⏰ {symbol} 分时数据 (最近10个点):")
        # timeline是DataFrame格式，取最后10行
        last_10 = timeline.tail(10)
        for index, row in last_10.iterrows():
            time_str = datetime.fromtimestamp(row['time']/1000).strftime('%H:%M:%S')
            print(f"  {time_str}: ${row['price']:.2f}, 成交量={row['volume']}")
    else:
        print("❌ 未获取到分时数据")
        
except Exception as e:
    print(f"❌ 获取分时数据失败: {e}")

⏰ AAPL 分时数据 (最近10个点):
  03:50:00: $229.52, 成交量=165229
  03:51:00: $229.65, 成交量=94571
  03:52:00: $229.65, 成交量=97829
  03:53:00: $229.65, 成交量=101918
  03:54:00: $229.88, 成交量=215142
  03:55:00: $229.73, 成交量=181368
  03:56:00: $229.73, 成交量=141324
  03:57:00: $229.81, 成交量=176779
  03:58:00: $229.74, 成交量=207317
  03:59:00: $229.65, 成交量=7943741


## 3. 账户信息接口

### 3.1 获取账户列表

In [11]:
# 获取账户列表
try:
    accounts = trade_client.get_managed_accounts()
    print("👤 账户信息:")
    if accounts:
        for account in accounts:
            print(f"  账户: {account}")
        
        # 优先选择模拟账户（PAPER类型）
        paper_account = None
        for account in accounts:
            if account.account_type == 'PAPER':
                paper_account = account
                break
        
        if paper_account:
            selected_account = paper_account
            print(f"✅ 选择模拟账户: {selected_account}")
        else:
            # 如果没有模拟账户，选择第一个账户
            selected_account = accounts[0]
            print(f"✅ 选择账户: {selected_account}")
    else:
        print("❌ 未找到可用账户")
        selected_account = None
        
except Exception as e:
    print(f"❌ 获取账户失败: {e}")
    selected_account = None

👤 账户信息:
  账户: AccountProfile({'account': '708815', 'capability': 'RegTMargin', 'status': 'Funded', 'account_type': 'STANDARD'})
  账户: AccountProfile({'account': 'U10015521', 'capability': 'CASH', 'status': 'Funded', 'account_type': 'GLOBAL'})
  账户: AccountProfile({'account': '21722764233480907', 'capability': 'RegTMargin', 'status': 'Funded', 'account_type': 'PAPER'})
✅ 选择模拟账户: AccountProfile({'account': '21722764233480907', 'capability': 'RegTMargin', 'status': 'Funded', 'account_type': 'PAPER'})


### 3.2 获取资产信息

In [12]:
# 获取资产信息
if selected_account:
    try:
        # 使用账户字符串而不是AccountProfile对象
        assets = trade_client.get_assets(account=selected_account.account)
        print("💰 资产信息:")
        
        if assets:
            for asset in assets:
                # 从summary获取资产信息
                if hasattr(asset, 'summary') and asset.summary:
                    summary = asset.summary
                    
                    # 显示基本信息
                    if hasattr(summary, 'currency'):
                        print(f"  货币: {summary.currency}")
                    
                    # 显示资产数据
                    if hasattr(summary, 'net_liquidation'):
                        print(f"  总资产: ${summary.net_liquidation:,.2f}")
                    if hasattr(summary, 'cash'):
                        print(f"  现金: ${summary.cash:,.2f}")
                    if hasattr(summary, 'buying_power'):
                        print(f"  可用资金: ${summary.buying_power:,.2f}")
                    if hasattr(summary, 'gross_position_value'):
                        print(f"  持仓市值: ${summary.gross_position_value:,.2f}")
                    if hasattr(summary, 'unrealized_pnl'):
                        print(f"  未实现盈亏: ${summary.unrealized_pnl:,.2f}")
                    if hasattr(summary, 'realized_pnl'):
                        print(f"  已实现盈亏: ${summary.realized_pnl:,.2f}")
                    if hasattr(summary, 'available_funds'):
                        print(f"  可用资金: ${summary.available_funds:,.2f}")
                    if hasattr(summary, 'excess_liquidity'):
                        print(f"  超额流动性: ${summary.excess_liquidity:,.2f}")
                else:
                    print("  无法获取资产摘要信息")
                
                print("---")
        else:
            print("  未获取到资产信息")
            
    except Exception as e:
        print(f"❌ 获取资产信息失败: {e}")
else:
    print("⚠️ 未选择账户，跳过资产查询")

💰 资产信息:
  货币: USD
  总资产: $598,552.54
  现金: $594,251.25
  可用资金: $2,386,374.71
  持仓市值: $inf
  未实现盈亏: $-49.43
  已实现盈亏: $-882.40
  可用资金: $inf
  超额流动性: $inf
---


### 3.3 获取持仓信息

In [13]:
# 获取持仓信息
if selected_account:
    try:
        # 使用账户字符串而不是AccountProfile对象
        positions = trade_client.get_positions(account=selected_account.account)
        print("📊 持仓信息:")
        
        if positions:
            for position in positions:
                print(f"  股票: {position.contract.symbol}")
                print(f"  数量: {position.quantity}")
                print(f"  平均成本: ${position.average_cost:.2f}")
                print(f"  市场价值: ${position.market_value:,.2f}")
                print(f"  未实现盈亏: ${position.unrealized_pnl:,.2f}")
                print("---")
        else:
            print("  暂无持仓")
            
    except Exception as e:
        print(f"❌ 获取持仓信息失败: {e}")
else:
    print("⚠️ 未选择账户，跳过持仓查询")

📊 持仓信息:
  股票: AAPL
  数量: 17
  平均成本: $232.56
  市场价值: $3,904.05
  未实现盈亏: $-49.43
---


### 3.4 获取交易成本

In [33]:
# 获取交易成本信息
def get_trading_costs(symbol, quantity, price, account=None, client=None):
    """获取交易成本估算
    
    Args:
        symbol: 股票代码
        quantity: 交易数量
        price: 交易价格
        account: 交易账户
        client: 交易客户端
    """
    # 获取交易客户端和账户
    if client is None:
        client = globals().get('trade_client')
    if account is None:
        account = globals().get('selected_account')
    
    if not client:
        print("❌ 交易客户端未初始化，请先运行客户端初始化代码")
        return None
        
    if not account:
        print("❌ 未选择账户，请先运行账户选择代码")
        return None
    
    try:
        from tigeropen.common.util.contract_utils import stock_contract
        from tigeropen.common.util.order_utils import limit_order
        
        # 创建合约对象
        contract = stock_contract(symbol=symbol, currency='USD')
        
        # 创建订单对象用于预览
        order = limit_order(account=account.account, contract=contract, 
                           action='BUY', limit_price=price, quantity=quantity)
        
        # 使用订单预览功能获取成本信息
        preview_result = client.preview_order(order)
        
        if preview_result:
            print(f"💰 {symbol} 交易成本预览:")
            print(f"  交易数量: {quantity}")
            print(f"  交易价格: ${price:.2f}")
            print(f"  交易金额: ${quantity * price:,.2f}")
            
            # 显示预览结果中的成本信息
            if hasattr(preview_result, 'commission'):
                print(f"  预估手续费: ${preview_result.commission:.2f}")
            if hasattr(preview_result, 'initial_margin'):
                print(f"  初始保证金: ${preview_result.initial_margin:.2f}")
            if hasattr(preview_result, 'maintenance_margin'):
                print(f"  维持保证金: ${preview_result.maintenance_margin:.2f}")
            if hasattr(preview_result, 'buying_power'):
                print(f"  所需购买力: ${preview_result.buying_power:.2f}")
            
            # 计算总成本
            commission = getattr(preview_result, 'commission', 0)
            total_cost = quantity * price + commission
            print(f"  总成本: ${total_cost:.2f}")
            
            return {
                'symbol': symbol,
                'quantity': quantity,
                'price': price,
                'trade_amount': quantity * price,
                'commission': commission,
                'total_cost': total_cost,
                'preview_result': preview_result
            }
        else:
            print("❌ 无法获取订单预览信息")
            return None
            
    except Exception as e:
        print(f"❌ 获取交易成本失败: {e}")
        # 提供基本的成本估算
        print(f"💡 基本成本估算:")
        print(f"  交易数量: {quantity}")
        print(f"  交易价格: ${price:.2f}")
        print(f"  交易金额: ${quantity * price:,.2f}")
        
        # 美股交易成本估算（基于常见费率）
        if symbol and len(symbol) <= 5:  # 假设是美股
            # 老虎证券美股交易费用参考
            commission_rate = 0.01  # 每股1美分，最低1美元
            commission = max(quantity * commission_rate, 1.0)
            sec_fee = quantity * price * 0.0000278  # SEC费用
            finra_fee = min(quantity * 0.000166, 8.5)  # FINRA费用
            
            total_fees = commission + sec_fee + finra_fee
            total_cost = quantity * price + total_fees
            
            print(f"  估算佣金: ${commission:.2f}")
            print(f"  SEC费用: ${sec_fee:.4f}")
            print(f"  FINRA费用: ${finra_fee:.4f}")
            print(f"  总费用: ${total_fees:.2f}")
            print(f"  总成本: ${total_cost:.2f}")
            
            return {
                'symbol': symbol,
                'quantity': quantity,
                'price': price,
                'trade_amount': quantity * price,
                'commission': commission,
                'sec_fee': sec_fee,
                'finra_fee': finra_fee,
                'total_fees': total_fees,
                'total_cost': total_cost
            }
        
        return None

# 演示获取交易成本
if selected_account:
    # 示例1: AAPL 100股
    print("📊 交易成本演示 1: AAPL 100股")
    cost_info = get_trading_costs('AAPL', 100, 230.0)
    print()
    
    # 示例2: TSLA 50股
    print("📊 交易成本演示 2: TSLA 50股")
    cost_info = get_trading_costs('TSLA', 50, 250.0)
    print()
    
    # 示例3: 小额交易
    print("📊 交易成本演示 3: SPY 10股 (小额交易)")
    cost_info = get_trading_costs('SPY', 10, 450.0)
else:
    print("⚠️ 未选择账户，跳过交易成本演示")

📊 交易成本演示 1: AAPL 100股
💰 AAPL 交易成本预览:
  交易数量: 100
  交易价格: $230.00
  交易金额: $23,000.00
  总成本: $23000.00

📊 交易成本演示 2: TSLA 50股
💰 TSLA 交易成本预览:
  交易数量: 50
  交易价格: $250.00
  交易金额: $12,500.00
  总成本: $12500.00

📊 交易成本演示 3: SPY 10股 (小额交易)
💰 SPY 交易成本预览:
  交易数量: 10
  交易价格: $450.00
  交易金额: $4,500.00
  总成本: $4500.00


In [34]:
# 获取历史交易的实际成本
def get_historical_trading_costs(days=30, account=None, client=None):
    """获取历史交易的实际成本统计
    
    Args:
        days: 查询天数
        account: 交易账户
        client: 交易客户端
    """
    # 获取交易客户端和账户
    if client is None:
        client = globals().get('trade_client')
    if account is None:
        account = globals().get('selected_account')
    
    if not client:
        print("❌ 交易客户端未初始化，请先运行客户端初始化代码")
        return None
        
    if not account:
        print("❌ 未选择账户，请先运行账户选择代码")
        return None
    
    try:
        from datetime import datetime, timedelta
        
        # 计算查询时间范围
        end_time = datetime.now()
        start_time = end_time - timedelta(days=days)
        
        # 获取历史订单
        orders = client.get_orders(account=account.account, 
                                  start_time=int(start_time.timestamp() * 1000),
                                  end_time=int(end_time.timestamp() * 1000))
        
        if not orders:
            print(f"📊 过去{days}天内无交易记录")
            return None
        
        print(f"📊 过去{days}天交易成本统计:")
        print(f"  总订单数: {len(orders)}")
        
        total_commission = 0
        total_trade_amount = 0
        filled_orders = 0
        
        for order in orders:
            if hasattr(order, 'filled') and order.filled > 0:
                filled_orders += 1
                commission = getattr(order, 'commission', 0)
                avg_price = getattr(order, 'avg_fill_price', 0)
                filled_qty = order.filled
                
                trade_amount = avg_price * filled_qty
                total_commission += commission
                total_trade_amount += trade_amount
                
                print(f"  订单 {order.id}:")
                print(f"    标的: {getattr(order.contract, 'symbol', 'N/A') if hasattr(order, 'contract') and order.contract else 'N/A'}")
                print(f"    成交数量: {filled_qty}")
                print(f"    成交价格: ${avg_price:.2f}")
                print(f"    交易金额: ${trade_amount:.2f}")
                print(f"    手续费: ${commission:.2f}")
                print(f"    费率: {(commission/trade_amount*100):.4f}%" if trade_amount > 0 else "    费率: N/A")
                print("    ---")
        
        if filled_orders > 0:
            avg_commission_rate = (total_commission / total_trade_amount * 100) if total_trade_amount > 0 else 0
            print(f"📈 汇总统计:")
            print(f"  成交订单数: {filled_orders}")
            print(f"  总交易金额: ${total_trade_amount:,.2f}")
            print(f"  总手续费: ${total_commission:.2f}")
            print(f"  平均费率: {avg_commission_rate:.4f}%")
            
            return {
                'total_orders': len(orders),
                'filled_orders': filled_orders,
                'total_trade_amount': total_trade_amount,
                'total_commission': total_commission,
                'avg_commission_rate': avg_commission_rate
            }
        else:
            print("  无成交订单")
            return None
            
    except Exception as e:
        print(f"❌ 获取历史交易成本失败: {e}")
        return None

# 演示获取历史交易成本
if selected_account:
    print("🔍 获取历史交易成本...")
    historical_costs = get_historical_trading_costs(days=30)
else:
    print("⚠️ 未选择账户，跳过历史交易成本查询")

🔍 获取历史交易成本...
📊 过去30天交易成本统计:
  总订单数: 100
  无成交订单


In [35]:
# 交易成本比较工具
def compare_trading_costs(trades_list):
    """比较不同交易的成本效率
    
    Args:
        trades_list: 交易列表，每个元素包含 (symbol, quantity, price)
    """
    print("📊 交易成本比较:")
    print(f"{'标的':<8} {'数量':<8} {'价格':<10} {'交易额':<12} {'估算费用':<10} {'费率':<8} {'总成本':<12}")
    print("-" * 80)
    
    total_trade_amount = 0
    total_fees = 0
    
    for symbol, quantity, price in trades_list:
        trade_amount = quantity * price
        
        # 美股交易费用估算
        commission = max(quantity * 0.01, 1.0)  # 每股1美分，最低1美元
        sec_fee = trade_amount * 0.0000278  # SEC费用
        finra_fee = min(quantity * 0.000166, 8.5)  # FINRA费用
        
        total_fee = commission + sec_fee + finra_fee
        fee_rate = (total_fee / trade_amount * 100) if trade_amount > 0 else 0
        total_cost = trade_amount + total_fee
        
        print(f"{symbol:<8} {quantity:<8} ${price:<9.2f} ${trade_amount:<11,.2f} ${total_fee:<9.2f} {fee_rate:<7.4f}% ${total_cost:<11,.2f}")
        
        total_trade_amount += trade_amount
        total_fees += total_fee
    
    print("-" * 80)
    overall_fee_rate = (total_fees / total_trade_amount * 100) if total_trade_amount > 0 else 0
    print(f"{'总计':<8} {'':<8} {'':<10} ${total_trade_amount:<11,.2f} ${total_fees:<9.2f} {overall_fee_rate:<7.4f}% ${total_trade_amount + total_fees:<11,.2f}")
    
    # 成本效率分析
    print("\n💡 成本效率分析:")
    if overall_fee_rate < 0.1:
        print("  ✅ 整体费率较低，成本控制良好")
    elif overall_fee_rate < 0.5:
        print("  ⚠️ 整体费率适中，可考虑优化交易规模")
    else:
        print("  ❌ 整体费率较高，建议增加单笔交易规模以降低成本")
    
    return {
        'total_trade_amount': total_trade_amount,
        'total_fees': total_fees,
        'overall_fee_rate': overall_fee_rate
    }

# 演示交易成本比较
print("🔍 交易成本比较演示:")
trades_to_compare = [
    ('AAPL', 100, 230.0),
    ('TSLA', 50, 250.0),
    ('MSFT', 75, 380.0),
    ('GOOGL', 25, 140.0),
    ('SPY', 200, 450.0)
]

comparison_result = compare_trading_costs(trades_to_compare)

🔍 交易成本比较演示:
📊 交易成本比较:
标的       数量       价格         交易额          估算费用       费率       总成本         
--------------------------------------------------------------------------------
AAPL     100      $230.00    $23,000.00   $1.66      0.0072 % $23,001.66  
TSLA     50       $250.00    $12,500.00   $1.36      0.0108 % $12,501.36  
MSFT     75       $380.00    $28,500.00   $1.80      0.0063 % $28,501.80  
GOOGL    25       $140.00    $3,500.00    $1.10      0.0315 % $3,501.10   
SPY      200      $450.00    $90,000.00   $4.54      0.0050 % $90,004.54  
--------------------------------------------------------------------------------
总计                           $157,500.00  $10.45     0.0066 % $157,510.45 

💡 成本效率分析:
  ✅ 整体费率较低，成本控制良好


In [36]:
# 滑点成本分析工具
def analyze_slippage_costs(symbol, quantity, expected_price, market_impact_bps=5, timing_risk_bps=3, account=None, client=None):
    """分析滑点成本
    
    Args:
        symbol: 股票代码
        quantity: 交易数量
        expected_price: 预期交易价格
        market_impact_bps: 市场冲击成本(基点)
        timing_risk_bps: 时机风险成本(基点)
        account: 交易账户
        client: 行情客户端
    """
    # 获取行情客户端
    if client is None:
        client = globals().get('quote_client')
    
    if not client:
        print("❌ 行情客户端未初始化，使用估算值")
        current_price = expected_price
        bid_ask_spread = expected_price * 0.001  # 估算0.1%的买卖价差
    else:
        try:
            # 获取实时报价
            stock_briefs = client.get_stock_briefs([symbol])
            if stock_briefs is not None and not stock_briefs.empty:
                current_price = stock_briefs.iloc[0]['latest_price']
                # 尝试获取买卖价差
                bid_price = stock_briefs.iloc[0].get('bid_price', current_price * 0.999)
                ask_price = stock_briefs.iloc[0].get('ask_price', current_price * 1.001)
                bid_ask_spread = ask_price - bid_price
            else:
                current_price = expected_price
                bid_ask_spread = expected_price * 0.001
        except Exception as e:
            print(f"⚠️ 获取实时报价失败: {e}，使用估算值")
            current_price = expected_price
            bid_ask_spread = expected_price * 0.001
    
    # 计算各项滑点成本
    trade_amount = quantity * expected_price
    
    # 1. 买卖价差成本 (Bid-Ask Spread)
    spread_cost = bid_ask_spread * quantity / 2  # 买入时支付一半价差
    
    # 2. 市场冲击成本 (Market Impact)
    market_impact_cost = trade_amount * market_impact_bps / 10000
    
    # 3. 时机风险成本 (Timing Risk)
    timing_risk_cost = trade_amount * timing_risk_bps / 10000
    
    # 4. 价格偏离成本 (Price Deviation)
    price_deviation = abs(current_price - expected_price)
    price_deviation_cost = price_deviation * quantity
    
    # 总滑点成本
    total_slippage = spread_cost + market_impact_cost + timing_risk_cost + price_deviation_cost
    slippage_bps = (total_slippage / trade_amount) * 10000 if trade_amount > 0 else 0
    
    print(f"📊 {symbol} 滑点成本分析:")
    print(f"  交易数量: {quantity:,}")
    print(f"  预期价格: ${expected_price:.2f}")
    print(f"  当前价格: ${current_price:.2f}")
    print(f"  交易金额: ${trade_amount:,.2f}")
    print()
    print(f"  💰 滑点成本明细:")
    print(f"    买卖价差成本: ${spread_cost:.2f} ({bid_ask_spread:.4f} * {quantity} / 2)")
    print(f"    市场冲击成本: ${market_impact_cost:.2f} ({market_impact_bps} bps)")
    print(f"    时机风险成本: ${timing_risk_cost:.2f} ({timing_risk_bps} bps)")
    print(f"    价格偏离成本: ${price_deviation_cost:.2f} (偏离 ${price_deviation:.4f})")
    print(f"    总滑点成本: ${total_slippage:.2f} ({slippage_bps:.1f} bps)")
    print()
    
    # 滑点影响分析
    if slippage_bps < 10:
        impact_level = "低"
        recommendation = "滑点成本较低，可以正常交易"
    elif slippage_bps < 25:
        impact_level = "中等"
        recommendation = "滑点成本适中，建议分批交易或选择流动性更好的时段"
    else:
        impact_level = "高"
        recommendation = "滑点成本较高，强烈建议分批交易或使用算法交易"
    
    print(f"  📈 滑点影响评估: {impact_level}")
    print(f"  💡 交易建议: {recommendation}")
    
    return {
        'symbol': symbol,
        'quantity': quantity,
        'expected_price': expected_price,
        'current_price': current_price,
        'trade_amount': trade_amount,
        'spread_cost': spread_cost,
        'market_impact_cost': market_impact_cost,
        'timing_risk_cost': timing_risk_cost,
        'price_deviation_cost': price_deviation_cost,
        'total_slippage': total_slippage,
        'slippage_bps': slippage_bps,
        'impact_level': impact_level,
        'recommendation': recommendation
    }

# 演示滑点成本分析
print("🔍 滑点成本分析演示:")
print()

# 示例1: 大额交易
print("📊 示例1: AAPL 大额交易 (1000股)")
slippage_analysis1 = analyze_slippage_costs('AAPL', 1000, 230.0, market_impact_bps=8, timing_risk_bps=5)
print()

# 示例2: 中等交易
print("📊 示例2: TSLA 中等交易 (200股)")
slippage_analysis2 = analyze_slippage_costs('TSLA', 200, 250.0, market_impact_bps=5, timing_risk_bps=3)
print()

# 示例3: 小额交易
print("📊 示例3: SPY 小额交易 (50股)")
slippage_analysis3 = analyze_slippage_costs('SPY', 50, 450.0, market_impact_bps=2, timing_risk_bps=1)

🔍 滑点成本分析演示:

📊 示例1: AAPL 大额交易 (1000股)
📊 AAPL 滑点成本分析:
  交易数量: 1,000
  预期价格: $230.00
  当前价格: $229.65
  交易金额: $230,000.00

  💰 滑点成本明细:
    买卖价差成本: $0.00 (0.0000 * 1000 / 2)
    市场冲击成本: $184.00 (8 bps)
    时机风险成本: $115.00 (5 bps)
    价格偏离成本: $350.00 (偏离 $0.3500)
    总滑点成本: $649.00 (28.2 bps)

  📈 滑点影响评估: 高
  💡 交易建议: 滑点成本较高，强烈建议分批交易或使用算法交易

📊 示例2: TSLA 中等交易 (200股)
📊 TSLA 滑点成本分析:
  交易数量: 200
  预期价格: $250.00
  当前价格: $340.84
  交易金额: $50,000.00

  💰 滑点成本明细:
    买卖价差成本: $0.00 (0.0000 * 200 / 2)
    市场冲击成本: $25.00 (5 bps)
    时机风险成本: $15.00 (3 bps)
    价格偏离成本: $18168.00 (偏离 $90.8400)
    总滑点成本: $18208.00 (3641.6 bps)

  📈 滑点影响评估: 高
  💡 交易建议: 滑点成本较高，强烈建议分批交易或使用算法交易

📊 示例3: SPY 小额交易 (50股)
📊 SPY 滑点成本分析:
  交易数量: 50
  预期价格: $450.00
  当前价格: $642.69
  交易金额: $22,500.00

  💰 滑点成本明细:
    买卖价差成本: $0.00 (0.0000 * 50 / 2)
    市场冲击成本: $4.50 (2 bps)
    时机风险成本: $2.25 (1 bps)
    价格偏离成本: $9634.50 (偏离 $192.6900)
    总滑点成本: $9641.25 (4285.0 bps)

  📈 滑点影响评估: 高
  💡 交易建议: 滑点成本较高，强烈建议分批交易或使用算法交易


In [ ]:
# 综合交易成本分析（交易费 + 滑点）
def comprehensive_trading_cost_analysis(symbol, quantity, price, market_impact_bps=5, timing_risk_bps=3):
    """综合交易成本分析
    
    Args:
        symbol: 股票代码
        quantity: 交易数量
        price: 交易价格
        market_impact_bps: 市场冲击成本(基点)
        timing_risk_bps: 时机风险成本(基点)
    """
    print(f"📊 {symbol} 综合交易成本分析:")
    print("=" * 60)
    
    trade_amount = quantity * price
    
    # 1. 交易费用分析
    print("💰 1. 交易费用 (佣金 + 监管费用):")
    commission = max(quantity * 0.01, 1.0)  # 每股1美分，最低1美元
    sec_fee = trade_amount * 0.0000278  # SEC费用
    finra_fee = min(quantity * 0.000166, 8.5)  # FINRA费用
    total_fees = commission + sec_fee + finra_fee
    fees_bps = (total_fees / trade_amount) * 10000 if trade_amount > 0 else 0
    
    print(f"  佣金: ${commission:.2f}")
    print(f"  SEC费用: ${sec_fee:.4f}")
    print(f"  FINRA费用: ${finra_fee:.4f}")
    print(f"  小计: ${total_fees:.2f} ({fees_bps:.1f} bps)")
    print()
    
    # 2. 滑点成本分析
    print("📈 2. 滑点成本 (市场冲击 + 时机风险):")
    bid_ask_spread = price * 0.001  # 估算0.1%的买卖价差
    spread_cost = bid_ask_spread * quantity / 2
    market_impact_cost = trade_amount * market_impact_bps / 10000
    timing_risk_cost = trade_amount * timing_risk_bps / 10000
    total_slippage = spread_cost + market_impact_cost + timing_risk_cost
    slippage_bps = (total_slippage / trade_amount) * 10000 if trade_amount > 0 else 0
    
    print(f"  买卖价差成本: ${spread_cost:.2f}")
    print(f"  市场冲击成本: ${market_impact_cost:.2f} ({market_impact_bps} bps)")
    print(f"  时机风险成本: ${timing_risk_cost:.2f} ({timing_risk_bps} bps)")
    print(f"  小计: ${total_slippage:.2f} ({slippage_bps:.1f} bps)")
    print()
    
    # 3. 总成本汇总
    total_cost = total_fees + total_slippage
    total_cost_bps = (total_cost / trade_amount) * 10000 if trade_amount > 0 else 0
    effective_price = price + (total_cost / quantity)
    
    print("📋 3. 总成本汇总:")
    print(f"  交易金额: ${trade_amount:,.2f}")
    print(f"  交易费用: ${total_fees:.2f} ({fees_bps:.1f} bps)")
    print(f"  滑点成本: ${total_slippage:.2f} ({slippage_bps:.1f} bps)")
    print(f"  总交易成本: ${total_cost:.2f} ({total_cost_bps:.1f} bps)")
    print(f"  有效交易价格: ${effective_price:.4f} (原价 + 成本/股)")
    print()
    
    # 4. 成本结构分析
    fees_percentage = (total_fees / total_cost * 100) if total_cost > 0 else 0
    slippage_percentage = (total_slippage / total_cost * 100) if total_cost > 0 else 0
    
    print("📊 4. 成本结构分析:")
    print(f"  交易费用占比: {fees_percentage:.1f}%")
    print(f"  滑点成本占比: {slippage_percentage:.1f}%")
    
    if fees_percentage > slippage_percentage:
        print("  💡 主要成本来源: 交易费用，建议增加单笔交易规模以摊薄固定成本")
    else:
        print("  💡 主要成本来源: 滑点成本，建议分批交易或选择流动性更好的时段")
    
    # 5. 优化建议
    print("\n🎯 5. 交易优化建议:")
    if total_cost_bps < 15:
        print("  ✅ 总成本较低，当前交易策略合理")
    elif total_cost_bps < 30:
        print("  ⚠️ 总成本适中，可考虑以下优化:")
        print("     - 选择流动性更好的交易时段")
        print("     - 适当增加单笔交易规模")
    else:
        print("  ❌ 总成本较高，强烈建议优化:")
        print("     - 使用算法交易分批执行")
        print("     - 选择流动性最好的交易时段")
        print("     - 考虑使用限价单而非市价单")
    
    return {
        'symbol': symbol,
        'quantity': quantity,
        'price': price,
        'trade_amount': trade_amount,
        'total_fees': total_fees,
        'total_slippage': total_slippage,
        'total_cost': total_cost,
        'total_cost_bps': total_cost_bps,
        'effective_price': effective_price,
        'fees_percentage': fees_percentage,
        'slippage_percentage': slippage_percentage
    }

# 演示综合交易成本分析
print("🔍 综合交易成本分析演示:")
print()

# 不同规模的交易成本对比
test_cases = [
    ('AAPL', 100, 230.0, 3, 2),    # 小额交易
    ('AAPL', 500, 230.0, 5, 3),    # 中等交易
    ('AAPL', 2000, 230.0, 8, 5),   # 大额交易
]

for i, (symbol, qty, px, market_bps, timing_bps) in enumerate(test_cases, 1):
    print(f"📊 案例 {i}: {symbol} {qty}股 @ ${px}")
    result = comprehensive_trading_cost_analysis(symbol, qty, px, market_bps, timing_bps)
    print("\n" + "=" * 80 + "\n")

🔍 综合交易成本分析演示:

📊 案例 1: AAPL 100股 @ $230.0
📊 AAPL 综合交易成本分析:
💰 1. 交易费用 (佣金 + 监管费用):
  佣金: $1.00
  SEC费用: $0.6394
  FINRA费用: $0.0166
  小计: $1.66 (0.7 bps)

📈 2. 滑点成本 (市场冲击 + 时机风险):
  买卖价差成本: $11.50
  市场冲击成本: $6.90 (3 bps)
  时机风险成本: $4.60 (2 bps)
  小计: $23.00 (10.0 bps)

📋 3. 总成本汇总:
  交易金额: $23,000.00
  交易费用: $1.66 (0.7 bps)
  滑点成本: $23.00 (10.0 bps)
  总交易成本: $24.66 (10.7 bps)
  有效交易价格: $230.2466 (原价 + 成本/股)

📊 4. 成本结构分析:
  交易费用占比: 6.7%
  滑点成本占比: 93.3%
  💡 主要成本来源: 滑点成本，建议分批交易或选择流动性更好的时段

🎯 5. 交易优化建议:
  ✅ 总成本较低，当前交易策略合理


📊 案例 2: AAPL 500股 @ $230.0
📊 AAPL 综合交易成本分析:
💰 1. 交易费用 (佣金 + 监管费用):
  佣金: $5.00
  SEC费用: $3.1970
  FINRA费用: $0.0830
  小计: $8.28 (0.7 bps)

📈 2. 滑点成本 (市场冲击 + 时机风险):
  买卖价差成本: $57.50
  市场冲击成本: $57.50 (5 bps)
  时机风险成本: $34.50 (3 bps)
  小计: $149.50 (13.0 bps)

📋 3. 总成本汇总:
  交易金额: $115,000.00
  交易费用: $8.28 (0.7 bps)
  滑点成本: $149.50 (13.0 bps)
  总交易成本: $157.78 (13.7 bps)
  有效交易价格: $230.3156 (原价 + 成本/股)

📊 4. 成本结构分析:
  交易费用占比: 5.2%
  滑点成本占比: 94.8%
  💡 主要成本来源: 滑点成本，建议分批交易或选择流动性更好的时

2025-08-13 10:53:23,360 - ERROR - command: ERROR
id: 0
code: 4001
msg: "kick out by a new connection"



## 4. 真实交易接口

⚠️ **警告**: 以下是真实交易操作，会产生实际的买卖订单！

### 4.1 下单前准备

In [17]:
# 交易参数设置
TRADE_SYMBOL = 'AAPL'  # 交易标的
TRADE_QUANTITY = 1     # 交易数量（小量测试）

# 获取当前价格作为参考
if selected_account:
    try:
        # 使用get_stock_briefs获取股票报价，返回DataFrame格式
        stock_briefs = quote_client.get_stock_briefs([TRADE_SYMBOL])
        if stock_briefs is not None and not stock_briefs.empty:
            current_price = stock_briefs.iloc[0]['latest_price']
        else:
            current_price = 0
            print(f"⚠️ 无法获取 {TRADE_SYMBOL} 的价格信息")
        print(f"📊 {TRADE_SYMBOL} 当前价格: ${current_price:.2f}")
        
        # 设置限价单价格（比当前价格低5%，避免立即成交）
        limit_price = round(current_price * 0.95, 2)
        print(f"💡 建议限价买入价格: ${limit_price:.2f} (比当前价格低5%)")
        
    except Exception as e:
        print(f"❌ 获取当前价格失败: {e}")
        current_price = None
        limit_price = None
else:
    print("⚠️ 未选择账户，跳过价格查询")

📊 AAPL 当前价格: $229.65
💡 建议限价买入价格: $218.17 (比当前价格低5%)


### 4.2 限价买入订单

In [18]:
# 下限价买入订单
if selected_account and limit_price:
    try:
        from tigeropen.common.util.contract_utils import stock_contract
        from tigeropen.common.util.order_utils import limit_order
        
        # 创建合约对象
        contract = stock_contract(symbol=TRADE_SYMBOL, currency='USD')
        
        # 创建限价买入订单
        order = limit_order(account=selected_account.account, contract=contract, action='BUY',
                          limit_price=limit_price, quantity=TRADE_QUANTITY)
        
        print(f"📝 准备下单:")
        print(f"  标的: {order.contract.symbol}")
        print(f"  操作: {order.action}")
        print(f"  数量: {order.quantity}")
        print(f"  限价: ${order.limit_price:.2f}")
        
        # 确认是否执行真实交易
        # 直接执行真实交易
        print("🚀 正在执行买入订单...")
        result = trade_client.place_order(order)
        if result:
            # 处理不同类型的返回值
            if hasattr(result, 'id'):
                order_id = result.id
            else:
                # 如果返回的是订单ID（整数）
                order_id = result
            print(f"✅ 订单提交成功！订单ID: {order_id}")
            buy_order_id = order_id
        else:
            print("❌ 订单提交失败")
            buy_order_id = None
            
    except Exception as e:
        print(f"❌ 下单失败: {e}")
        buy_order_id = None
else:
    print("⚠️ 跳过下单操作")
    buy_order_id = None

📝 准备下单:
  标的: AAPL
  操作: BUY
  数量: 1
  限价: $218.17
🚀 正在执行买入订单...
✅ 订单提交成功！订单ID: 40127931473462272


### 4.3 查询订单状态

In [19]:
# 查询订单状态
if buy_order_id:
    try:
        order_status = trade_client.get_order(id=buy_order_id)
        print(f"📋 订单详细信息:")
        print(f"  订单ID: {order_status.id}")
        print(f"  外部订单ID: {getattr(order_status, 'order_id', 'N/A')}")
        print(f"  股票代码: {getattr(order_status.contract, 'symbol', 'N/A') if hasattr(order_status, 'contract') and order_status.contract else 'N/A'}")
        print(f"  股票名称: {getattr(order_status.contract, 'local_symbol', 'N/A') if hasattr(order_status, 'contract') and order_status.contract else 'N/A'}")
        print(f"  订单状态: {order_status.status}")
        print(f"  订单类型: {getattr(order_status, 'order_type', 'N/A')}")
        print(f"  交易方向: {getattr(order_status, 'action', 'N/A')}")
        print(f"  订单数量: {getattr(order_status, 'quantity', 'N/A')}")
        print(f"  已成交数量: {order_status.filled}")
        print(f"  剩余数量: {order_status.remaining}")
        print(f"  限价价格: ${getattr(order_status, 'limit_price', 0):.2f}")
        print(f"  平均成交价: ${getattr(order_status, 'avg_fill_price', 0):.2f}")
        print(f"  订单时间: {getattr(order_status, 'order_time', 'N/A')}")
        print(f"  更新时间: {getattr(order_status, 'update_time', 'N/A')}")
        print(f"  手续费: ${getattr(order_status, 'commission', 0):.2f}")
        print(f"  币种: {getattr(order_status.contract, 'currency', 'N/A') if hasattr(order_status, 'contract') and order_status.contract else 'N/A'}")
        print(f"  市场: {getattr(order_status.contract, 'market', 'N/A') if hasattr(order_status, 'contract') and order_status.contract else 'N/A'}")
        print(f"  有效期: {getattr(order_status, 'time_in_force', 'N/A')}")
        print(f"  盘前盘后: {getattr(order_status, 'outside_rth', 'N/A')}")
        print(f"  订单来源: {getattr(order_status, 'source', 'N/A')}")
        if hasattr(order_status, 'remark') and order_status.remark:
            print(f"  备注信息: {order_status.remark}")
        
    except Exception as e:
        print(f"❌ 查询订单状态失败: {e}")
else:
    print("⚠️ 无订单ID，跳过状态查询")

📋 订单详细信息:
  订单ID: 40127931473462272
  外部订单ID: 2649
  股票代码: AAPL
  股票名称: None
  订单状态: OrderStatus.EXPIRED
  订单类型: LMT
  交易方向: BUY
  订单数量: 1
  已成交数量: 0
  剩余数量: 1
  限价价格: $218.17
  平均成交价: $0.00
  订单时间: 1755051020000
  更新时间: 1755051020000
  手续费: $0.00
  币种: USD
  市场: US
  有效期: DAY
  盘前盘后: True
  订单来源: openapi


### 4.4 获取订单列表

In [20]:
# 获取最近订单列表
if selected_account:
    try:
        orders = trade_client.get_orders(account=selected_account.account)
        print(f"📋 最近订单列表 (最多显示5个):")
        
        for i, order in enumerate(orders[:5]):
            print(f"  订单{i+1}:")
            print(f"    ID: {order.id}")
            print(f"    标的: {order.contract.symbol}")
            print(f"    操作: {order.action}")
            print(f"    数量: {order.quantity}")
            print(f"    状态: {order.status}")
            print(f"    时间: {datetime.fromtimestamp(order.order_time/1000)}")
            print("    ---")
            
    except Exception as e:
        print(f"❌ 获取订单列表失败: {e}")
else:
    print("⚠️ 未选择账户，跳过订单查询")

📋 最近订单列表 (最多显示5个):
  订单1:
    ID: 40127931473462272
    标的: AAPL
    操作: BUY
    数量: 1
    状态: OrderStatus.EXPIRED
    时间: 2025-08-13 10:10:20
    ---
  订单2:
    ID: 40127926314469376
    标的: QQQ
    操作: BUY
    数量: 1
    状态: OrderStatus.EXPIRED
    时间: 2025-08-13 10:09:41
    ---
  订单3:
    ID: 40127926294151168
    标的: TSLA
    操作: BUY
    数量: 1
    状态: OrderStatus.EXPIRED
    时间: 2025-08-13 10:09:41
    ---
  订单4:
    ID: 40127926275539968
    标的: AAPL
    操作: BUY
    数量: 1
    状态: OrderStatus.EXPIRED
    时间: 2025-08-13 10:09:41
    ---
  订单5:
    ID: 40127920857024512
    标的: AAPL
    操作: BUY
    数量: 1
    状态: OrderStatus.EXPIRED
    时间: 2025-08-13 10:08:59
    ---


### 4.5 撤销订单

In [21]:
# 撤销订单
if buy_order_id:
    try:
        # 确认是否撤销
        # 直接执行撤单操作
        print(f"🚀 正在撤销订单 {buy_order_id}...")
        cancel_result = trade_client.cancel_order(id=buy_order_id)
        if cancel_result:
            print(f"✅ 订单撤销成功！")
        else:
            print("❌ 订单撤销失败")
            
    except Exception as e:
        print(f"❌ 撤销订单失败: {e}")
else:
    print("⚠️ 无订单ID，跳过撤销操作")

🚀 正在撤销订单 40127931473462272...
✅ 订单撤销成功！


### 4.6 市价单示例

In [22]:
# 市价单示例（谨慎使用）
def place_market_order(symbol, action, quantity, account=None, client=None):
    """下市价单"""
    # 获取交易客户端和账户
    if client is None:
        client = globals().get('trade_client')
    if account is None:
        account = globals().get('selected_account')
    
    if not client:
        print("❌ 交易客户端未初始化，请先运行客户端初始化代码")
        return None
        
    if not account:
        print("❌ 未选择账户，请先运行账户选择代码")
        return None
        
    try:
        from tigeropen.trade.domain.order import Order
        from tigeropen.common.consts import SecurityType, OrderType
        from tigeropen.common.util.contract_utils import stock_contract
        
        # 创建合约对象
        contract = stock_contract(symbol=symbol, currency='USD')
        
        # 创建订单对象（使用正确的构造函数参数）
        order = Order(account=account.account, contract=contract, action=action)
        order.order_type = 'MKT'
        order.quantity = quantity
        order.time_in_force = 'DAY'
        
        print(f"📝 市价单准备:")
        print(f"  标的: {symbol}")
        print(f"  操作: {action}")
        print(f"  数量: {quantity}")
        print(f"  类型: 市价单")
        
        print("🚀 正在执行市价单...")
        result = trade_client.place_order(order)
        if result:
            # 处理不同类型的返回值
            if hasattr(result, 'id'):
                order_id = result.id
            else:
                # 如果返回的是订单ID（整数）
                order_id = result
            print(f"✅ 市价单提交成功！订单ID: {order_id}")
            return order_id
        else:
            print("❌ 市价单提交失败")
            return None
            
    except Exception as e:
        print(f"❌ 市价单下单失败: {e}")
        return None

# 示例调用
market_order_id = place_market_order('AAPL', 'BUY', 1)
print("💡 市价单函数已定义，需要时可以调用")

📝 市价单准备:
  标的: AAPL
  操作: BUY
  数量: 1
  类型: 市价单
🚀 正在执行市价单...
❌ 市价单下单失败: code=1200 msg=standard account response error(bad_request:当前时段不支持下单)
💡 市价单函数已定义，需要时可以调用


## 5. 期权数据接口

### 5.1 获取期权链

In [23]:
# 获取期权链
option_symbol = 'AAPL'

try:
    # 获取期权到期日
    expiry_dates = quote_client.get_option_expirations(symbols=[option_symbol])
    
    if expiry_dates is not None and len(expiry_dates) > 0:
        print(f"📅 {option_symbol} 期权到期日 (前5个):")
        # expiry_dates 是 DataFrame 格式，需要正确处理
        for i, row in expiry_dates.head(5).iterrows():
            # 根据实际返回的列名获取日期
            date = row.get('date', row.get('expiry', row.get('expiration_date', str(row))))
            print(f"  {i+1}. {date}")
            
        # 选择最近的到期日
        # 从DataFrame中获取第一行的日期数据
        first_row = expiry_dates.iloc[0]
        nearest_expiry = first_row.get('date', first_row.get('expiry', first_row.get('expiration_date', str(first_row))))
        print(f"✅ 选择到期日: {nearest_expiry}")
        
    else:
        print(f"❌ 未找到 {option_symbol} 的期权到期日")
        nearest_expiry = None
        
except Exception as e:
    print(f"❌ 获取期权到期日失败: {e}")
    nearest_expiry = None

📅 AAPL 期权到期日 (前5个):
  1. 2025-08-15
  2. 2025-08-22
  3. 2025-08-29
  4. 2025-09-05
  5. 2025-09-12
✅ 选择到期日: 2025-08-15


In [24]:
# 获取期权链数据
if nearest_expiry:
    try:
        option_chain = quote_client.get_option_chain(
            symbol=option_symbol,
            expiry=nearest_expiry
        )
        if option_chain is not None:
            # option_chain是DataFrame，需要按put_call列筛选
            calls = option_chain[option_chain['put_call'] == 'CALL'] if 'put_call' in option_chain.columns else pd.DataFrame()
            puts = option_chain[option_chain['put_call'] == 'PUT'] if 'put_call' in option_chain.columns else pd.DataFrame()
            
            print(f"  看涨期权数量: {len(calls)}")
            print(f"  看跌期权数量: {len(puts)}")
            print(f"  总期权数量: {len(option_chain)}")
            
            if len(calls) > 0:
                print("  看涨期权 (前3个):")
                for i, (idx, call) in enumerate(calls.head(3).iterrows()):
                    print(f"    {i+1}. {call['symbol']} - 行权价: ${call['strike']} - 最新价: ${call['latest_price']}")
            
            if len(puts) > 0:
                print("  看跌期权 (前3个):")
                for i, (idx, put) in enumerate(puts.head(3).iterrows()):
                    print(f"    {i+1}. {put['symbol']} - 行权价: ${put['strike']} - 最新价: ${put['latest_price']}")
        else:
            print("❌ 未获取到期权链数据")
            
    except Exception as e:
        print(f"❌ 获取期权链失败: {e}")
else:
    print("⚠️ 无到期日信息，跳过期权链查询")

  看涨期权数量: 72
  看跌期权数量: 72
  总期权数量: 144
  看涨期权 (前3个):
    1. AAPL - 行权价: $90.0 - 最新价: $138.6
    2. AAPL - 行权价: $95.0 - 最新价: $134.48
    3. AAPL - 行权价: $100.0 - 最新价: $127.65
  看跌期权 (前3个):
    1. AAPL - 行权价: $90.0 - 最新价: $0.01
    2. AAPL - 行权价: $95.0 - 最新价: $0.01
    3. AAPL - 行权价: $100.0 - 最新价: $0.01


### 5.2 获取期权报价

In [25]:
# 获取期权报价
if nearest_expiry:
    try:
        # 从期权链中获取实际的期权标识符
        if option_chain is not None and not option_chain.empty:
            # 获取看涨和看跌期权
            calls = option_chain[option_chain['put_call'] == 'CALL']
            puts = option_chain[option_chain['put_call'] == 'PUT']
            
            if not calls.empty and not puts.empty:
                # 选择接近当前价格的期权（例如第10个）
                call_idx = min(9, len(calls) - 1)  # 防止索引越界
                put_idx = min(9, len(puts) - 1)
                
                call_symbol = calls.iloc[call_idx]['identifier']
                put_symbol = puts.iloc[put_idx]['identifier']
                
                print(f"📊 期权合约:")
                print(f"  看涨: {call_symbol}")
                print(f"  看跌: {put_symbol}")
                
                # 获取期权报价
                option_quotes = quote_client.get_option_briefs([call_symbol, put_symbol])
            else:
                print("❌ 期权链中没有找到看涨或看跌期权")
                option_quotes = None
        else:
            print("❌ 期权链数据为空")
            option_quotes = None
        
        # 处理期权报价结果
        if option_quotes is not None and not option_quotes.empty:
            print("💰 期权报价:")
            for idx, quote in option_quotes.iterrows():
                print(f"  {quote['identifier']}:")
                # 安全格式化数值字段
                latest_price = float(quote['latest_price']) if quote['latest_price'] not in [None, '', 'N/A'] else 0.0
                bid_price = float(quote['bid_price']) if quote['bid_price'] not in [None, '', 'N/A'] else 0.0
                ask_price = float(quote['ask_price']) if quote['ask_price'] not in [None, '', 'N/A'] else 0.0
                strike = float(quote['strike']) if quote['strike'] not in [None, '', 'N/A'] else 0.0
                volume = int(quote['volume']) if quote['volume'] not in [None, '', 'N/A'] else 0
                
                print(f"    最新价: ${latest_price:.2f}")
                print(f"    买价: ${bid_price:.2f}")
                print(f"    卖价: ${ask_price:.2f}")
                print(f"    成交量: {volume}")
                print(f"    行权价: ${strike:.2f}")
                print(f"    期权类型: {quote['put_call']}")
                print("    ---")
        else:
            print("❌ 未获取到期权报价")
            
    except Exception as e:
        print(f"❌ 获取期权报价失败: {e}")
else:
    print("⚠️ 无到期日信息，跳过期权报价查询")

📊 期权合约:
  看涨: AAPL  250815C00135000
  看跌: AAPL  250815P00135000
💰 期权报价:
  AAPL  250815C00135000:
    最新价: $91.45
    买价: $93.95
    卖价: $95.30
    成交量: 0
    行权价: $135.00
    期权类型: CALL
    ---
  AAPL  250815P00135000:
    最新价: $0.01
    买价: $nan
    卖价: $0.01
    成交量: 0
    行权价: $135.00
    期权类型: PUT
    ---


## 6. 推送服务接口

### 6.1 实时报价推送

In [ ]:
# 实时报价推送示例
def on_quote_changed(quote_data):
    """报价推送回调函数"""
    print(f"📡 收到报价推送数据: {type(quote_data)}")
    try:
        # 尝试解析推送数据
        if hasattr(quote_data, 'symbol'):
            print(f"  股票代码: {quote_data.symbol}")
        if hasattr(quote_data, 'latest_price') and quote_data.latest_price:
            print(f"  最新价格: ${quote_data.latest_price:.2f}")
        if hasattr(quote_data, 'volume') and quote_data.volume:
            print(f"  成交量: {quote_data.volume}")
        if hasattr(quote_data, 'timestamp'):
            print(f"  时间戳: {quote_data.timestamp}")
    except Exception as e:
        print(f"  解析数据时出错: {e}")
        print(f"  原始数据: {quote_data}")

def start_quote_push(symbols, duration=30):
    """启动报价推送"""
    try:
        # 连接推送服务
        push_client.connect(client_config.tiger_id, client_config.private_key)
        
        # 绑定回调函数
        push_client.quote_changed = on_quote_changed
        
        # 订阅报价推送（不需要callback参数）
        push_client.subscribe_quote(symbols=symbols)
        print(f"📡 开始推送 {symbols} 报价，持续 {duration} 秒...")
        
        # 等待推送数据
        time.sleep(duration)
        
        # 取消订阅
        push_client.unsubscribe_quote(symbols=symbols)
        print("📡 报价推送已停止")
        
        # 断开连接
        push_client.disconnect()
        
    except Exception as e:
        print(f"❌ 报价推送失败: {e}")

# 示例调用
if push_client is not None:
    print("🚀 启动实时报价推送演示...")
    start_quote_push(['AAPL', 'TSLA'], 10)
else:
    print("⚠️ 推送客户端未初始化，无法启动报价推送")
    print("💡 报价推送函数已定义，需要时可以调用")

2025-08-13 10:07:46,772 - INFO - established connection to host openapi.tigerfintech.com, port 9883


🚀 启动实时报价推送演示...
📡 开始推送 ['AAPL', 'TSLA'] 报价，持续 10 秒...
📡 报价推送已停止


2025-08-13 10:07:56,951 - INFO - established connection to host openapi.tigerfintech.com, port 9883


### 6.2 订单状态推送

In [ ]:
# 订单状态推送示例
def on_order_changed(order_data):
    """订单推送回调函数"""
    print(f"📋 收到订单推送: {order_data}")
    if hasattr(order_data, 'symbol'):
        print(f"  标的: {order_data.symbol}")
    if hasattr(order_data, 'status'):
        print(f"  状态: {order_data.status}")

def start_order_push(duration=60, account=None):
    """启动订单推送"""
    # 获取账户
    if account is None:
        account = globals().get('selected_account')
    
    if not account:
        print("❌ 未选择账户，请先运行账户选择代码")
        return
        
    try:
        # 连接推送服务
        push_client.connect(client_config.tiger_id, client_config.private_key)
        
        # 绑定回调函数
        push_client.order_changed = on_order_changed
        
        # 订阅订单推送（不需要callback参数）
        push_client.subscribe_order(account.account)
        print(f"📋 开始推送订单状态，持续 {duration} 秒...")
        
        # 等待推送数据
        time.sleep(duration)
        
        # 取消订阅
        push_client.unsubscribe_order()
        print("📋 订单推送已停止")
        
        # 断开连接
        push_client.disconnect()
        
    except Exception as e:
        print(f"❌ 订单推送失败: {e}")

# 示例调用
if push_client is not None:
    print("🚀 启动订单状态推送演示...")
    start_order_push(30)
else:
    print("⚠️ 推送客户端未初始化，无法启动订单推送")
    print("💡 订单推送函数已定义，需要时可以调用")

2025-08-13 10:07:56,941 - INFO - established connection to host openapi.tigerfintech.com, port 9883


🚀 启动订单状态推送演示...
📋 开始推送订单状态，持续 30 秒...


2025-08-13 10:07:57,046 - ERROR - command: ERROR
id: 0
code: 4001
msg: "kick out by a new connection"



📋 订单推送已停止


2025-08-13 10:08:27,095 - INFO - established connection to host openapi.tigerfintech.com, port 9883


## 7. 高级功能演示

### 7.1 批量下单

In [28]:
# 批量下单示例
def batch_place_orders(order_configs, account=None, client=None):
    """批量下单
    
    Args:
        order_configs: 订单配置列表，每个配置包含symbol, action, quantity, price等
        account: 交易账户，如果为None则自动获取
        client: 交易客户端，如果为None则自动获取
    """
    # 获取交易客户端和账户
    if client is None:
        client = globals().get('trade_client')
    if account is None:
        account = globals().get('selected_account')
    
    if not client:
        print("❌ 交易客户端未初始化，请先运行客户端初始化代码")
        return []
        
    if not account:
        print("❌ 未选择账户，请先运行账户选择代码")
        return []
        
    order_ids = []
    
    for i, config in enumerate(order_configs):
        try:
            from tigeropen.trade.domain.order import Order
            from tigeropen.common.consts import SecurityType, OrderType
            from tigeropen.common.util.contract_utils import stock_contract
            
            # 创建合约对象
            contract = stock_contract(symbol=config['symbol'], currency='USD')
            
            # 创建订单对象（使用正确的构造函数参数）
            order = Order(account=account.account, contract=contract, action=config['action'])
            order.order_type = 'LMT'
            order.quantity = config['quantity']
            order.limit_price = round(config['price'], 2)
            order.time_in_force = 'DAY'
            
            print(f"📝 批量订单 {i+1}: {config['symbol']} {config['action']} {config['quantity']}@${config['price']:.2f}")
            
            # 执行批量订单
            result = trade_client.place_order(order)
            if result:
                # 处理不同类型的返回值
                if hasattr(result, 'id'):
                    order_id = result.id
                else:
                    # 如果返回的是订单ID（整数）
                    order_id = result
                order_ids.append(order_id)
            
        except Exception as e:
            print(f"❌ 批量订单 {i+1} 失败: {e}")
            
    return order_ids

# 示例配置
sample_orders = [
    {'symbol': 'AAPL', 'action': 'BUY', 'quantity': 1, 'price': 150.0},
    {'symbol': 'TSLA', 'action': 'BUY', 'quantity': 1, 'price': 200.0},
    {'symbol': 'QQQ', 'action': 'BUY', 'quantity': 1, 'price': 300.0}
]

print("💡 批量下单示例:")
batch_place_orders(sample_orders)

💡 批量下单示例:
📝 批量订单 1: AAPL BUY 1@$150.00
📝 批量订单 2: TSLA BUY 1@$200.00
📝 批量订单 3: QQQ BUY 1@$300.00


[40127936889749504, 40127936909150208, 40127936928940032]

### 7.2 智能交易策略

In [29]:
# 简单的均值回归策略示例
import time
import numpy as np
import pandas as pd

def mean_reversion_strategy(symbol, lookback_days=20, threshold=0.02, quote_client=None):
    """均值回归策略
    
    Args:
        symbol: 交易标的
        lookback_days: 回看天数
        threshold: 偏离阈值
    """
    try:
        # 获取历史数据 - 确保获取足够的交易日数据
        end_time = int(time.time() * 1000)
        # 考虑到周末和节假日，使用更大的时间范围确保获取足够数据
        begin_time = end_time - lookback_days * 3 * 24 * 60 * 60 * 1000  # 扩大到3倍时间范围
        
        bars = quote_client.get_bars(
            symbols=[symbol],
            period=BarPeriod.DAY,
            begin_time=begin_time,
            end_time=end_time,
            right=QuoteRight.BR
        )
        
        if bars is None:
            print(f"❌ {symbol} 无法获取历史数据")
            return None
            
        # 计算均价和当前价格
        if isinstance(bars, pd.DataFrame):
            prices = bars['close'].values[-lookback_days:]  # 取最近lookback_days天的数据
        else:
            prices = [bar.close for bar in bars[-lookback_days:]]
        
        mean_price = np.mean(prices)
        current_price = prices[-1]
        
        # 计算偏离度
        deviation = (current_price - mean_price) / mean_price
        
        print(f"📊 {symbol} 策略分析:")
        print(f"  当前价格: ${current_price:.2f}")
        print(f"  {lookback_days}日均价: ${mean_price:.2f}")
        print(f"  偏离度: {deviation:.2%}")
        
        # 生成交易信号和预期盈利计算
        expected_profit = 0
        profit_percentage = 0
        
        if deviation < -threshold:
            signal = 'BUY'
            # 买入预期：价格回归到均值的盈利
            expected_profit = mean_price - current_price
            profit_percentage = expected_profit / current_price
            print(f"📈 交易信号: 买入 (价格低于均值 {threshold:.1%})")
            print(f"💰 预期盈利: ${expected_profit:.2f} ({profit_percentage:.2%})")
        elif deviation > threshold:
            signal = 'SELL'
            # 卖出预期：价格回归到均值的盈利（做空收益）
            expected_profit = current_price - mean_price
            profit_percentage = expected_profit / current_price
            print(f"📉 交易信号: 卖出 (价格高于均值 {threshold:.1%})")
            print(f"💰 预期盈利: ${expected_profit:.2f} ({profit_percentage:.2%})")
        else:
            signal = 'HOLD'
            print(f"⏸️ 交易信号: 持有 (价格在正常范围内)")
            
        return {
            'symbol': symbol,
            'signal': signal,
            'current_price': current_price,
            'mean_price': mean_price,
            'deviation': deviation,
            'expected_profit': expected_profit,
            'profit_percentage': profit_percentage
        }
        
    except Exception as e:
        print(f"❌ 策略分析失败: {e}")
        return None

# 运行策略
strategy_result = mean_reversion_strategy('AAPL', 20, 0.02, quote_client)

📊 AAPL 策略分析:
  当前价格: $229.65
  20日均价: $213.50
  偏离度: 7.56%
📉 交易信号: 卖出 (价格高于均值 2.0%)
💰 预期盈利: $16.15 (7.03%)


### 7.3 风险管理

In [30]:
# 风险管理示例
def risk_check(symbol, quantity, price, max_position_pct=0.1, max_single_trade_pct=0.05, account=None, client=None):
    """风险检查
    
    Args:
        symbol: 交易标的
        quantity: 交易数量
        price: 交易价格
        max_position_pct: 最大持仓比例
        max_single_trade_pct: 单笔交易最大比例
        account: 交易账户，如果为None则自动获取
        client: 交易客户端，如果为None则自动获取
    """
    # 获取交易客户端和账户
    if client is None:
        client = globals().get('trade_client')
    if account is None:
        account = globals().get('selected_account')
    
    if not client:
        print("❌ 交易客户端未初始化，请先运行客户端初始化代码")
        return False
        
    if not account:
        print("❌ 未选择账户，请先运行账户选择代码")
        return False
        
    try:
        # 获取账户资产
        assets = trade_client.get_assets(account=account.account)
        if not assets:
            print("❌ 无法获取资产信息")
            return False
            
        # 从summary获取总资产
        if hasattr(assets[0], 'summary') and assets[0].summary and hasattr(assets[0].summary, 'net_liquidation'):
            total_value = assets[0].summary.net_liquidation
        else:
            print("❌ 无法获取总资产信息")
            return False
        trade_value = quantity * price
        
        # 检查单笔交易比例
        single_trade_pct = trade_value / total_value
        if single_trade_pct > max_single_trade_pct:
            print(f"❌ 风险检查失败: 单笔交易比例 {single_trade_pct:.2%} 超过限制 {max_single_trade_pct:.2%}")
            return False
            
        # 获取当前持仓
        positions = trade_client.get_positions(account=account.account)
        current_position = 0
        
        for position in positions:
            if position.contract.symbol == symbol:
                current_position = position.quantity
                break
                
        # 计算交易后持仓
        new_position_value = (current_position + quantity) * price
        position_pct = new_position_value / total_value
        
        if position_pct > max_position_pct:
            print(f"❌ 风险检查失败: 持仓比例 {position_pct:.2%} 超过限制 {max_position_pct:.2%}")
            return False
            
        print(f"✅ 风险检查通过:")
        print(f"  单笔交易比例: {single_trade_pct:.2%}")
        print(f"  交易后持仓比例: {position_pct:.2%}")
        return True
        
    except Exception as e:
        print(f"❌ 风险检查失败: {e}")
        return False

# 示例风险检查
if selected_account:
    risk_check('AAPL', 10, 150.0)
else:
    print("⚠️ 未选择账户，跳过风险检查")

✅ 风险检查通过:
  单笔交易比例: 0.25%
  交易后持仓比例: 0.68%


## 总结

本Notebook演示了老虎证券API的完整功能，包括:

### ✅ 已演示的功能
1. **环境配置**: API连接和认证
2. **行情数据**: 实时报价、K线数据、分时数据
3. **账户管理**: 账户信息、资产查询、持仓管理
4. **真实交易**: 限价单、市价单、订单管理
5. **期权数据**: 期权链、期权报价
6. **推送服务**: 实时数据推送
7. **高级功能**: 批量交易、策略分析、风险管理

### ⚠️ 重要提醒
- 本演示包含**真实交易**操作，请谨慎使用
- 建议先在模拟环境中测试
- 交易前请确保充分理解风险
- 遵守相关法律法规和交易规则

### 📚 参考资料
- [老虎证券API文档](https://quant.itiger.com/openapi)
- [API接口整理文档](../老虎证券API接口整理.md)
- [数据获取工具](../data_fetcher.py)

## 使用说明

### 1. 环境准备
```bash
# 安装依赖
pip install tigeropen pandas numpy
```

### 2. 配置文件
确保以下文件存在并正确配置:
- `../config/tiger_openapi_config.properties`
- `../config/private_key.pem`

### 3. 运行步骤
1. 按顺序执行每个cell
2. 仔细阅读每个操作的说明
3. 真实交易前请三思而后行
4. 建议先用小额资金测试

### 4. 安全建议
- 不要在生产环境中运行未经测试的代码
- 设置合理的风险控制参数
- 定期检查账户状态
- 保护好API密钥和私钥文件